In [1]:
# Tensorflow v.2.3.1
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    Input,
    MaxPooling2D,
)
from tensorflow.keras import Model
import tensorflow as tf
import typing

In [2]:
tf.config.run_functions_eagerly(True)

In [ ]:
# By setting run_functions_eagerly to True, you are telling TensorFlow to execute functions eagerly, rather than building a computation graph and executing it later. This can be useful for:
# Debugging: Eager execution can make it easier to debug your code, since you can see the results of each operation immediately.
# Rapid prototyping: Eager execution can speed up the development process, since you don't need to build a computation graph before executing your code.

In [3]:
def make_conv_layer(
    X: tf.Tensor,
    architecture: typing.List[ typing.Union[int, str] ],  #Type Hint:
    # The type hint indicates that the architecture parameter should be a list (typing.List) containing elements that can be either integers (int) or strings (str).
    # Union Type:
    # The typing.Union[int, str] syntax specifies a union type, which means that the elements in the list can be of either type int or type str.
    activation: str = 'relu'
) -> tf.Tensor:
    """
    Method to create convolution layers for VGGNet.
    In VGGNet
        - Kernal is always 3x3 for conv-layer with padding 1 and stride 1.
        - 2x2 kernel for max pooling with stride of 2.

    Arguments:
    X            -- input tensor
    architecture -- number of output channel per convolution layers in VGGNet
    activation   -- type of activation method

    Returns:
    X           -- output tensor
    """

    # architecture: typing.List[typing.Union[int, str]] = [
    # 64,  # integer
    # "relu",  # string
    # 128,  # integer
    # "max_pool"  # string
    # ]

    for output in architecture:

        # convolution layer
        if type(output) == int:
            out_channels = output

            X = Conv2D(
                filters = out_channels,
                kernel_size = (3, 3),
                strides = (1, 1),
                padding = "same"
            )(X)
            X = BatchNormalization()(X) #Batch normalization is a technique used in deep learning to normalize the inputs 
            # to each layer of a neural network. 
            # Calculate the mean and variance: For each mini-batch, calculate the mean and variance of the input data.
            # Normalize the input: Normalize the input data by subtracting the mean and dividing by the standard deviation (square root of variance).
            # Scale and shift: Scale and shift the normalized input using learnable parameters (gamma and beta).
            X = Activation(activation)(X)

            # relu activation is added (by default activation) so that all the
            # negative values are not passed to the next layer

        # max-pooling layer
        else:
            X = MaxPooling2D(
                pool_size = (2, 2),
                strides = (2, 2)
            )(X)

    return X

In [4]:
def make_dense_layer(X: tf.Tensor, output_units: int, dropout = 0.5, activation = 'relu') -> tf.Tensor:
    """
    Method to create dense layer for VGGNet.

    Arguments:
    X            -- input tensor
    output_units -- output tensor size
    dropout      -- dropout value for regularization
    activation   -- type of activation method

    Returns:
    X            -- input tensor
    """

    X = Dense(units = output_units)(X)
    X = BatchNormalization()(X)
    X = Activation(activation)(X)
    X = Dropout(dropout)(X)

    return X

In [ ]:
# @tf.function is a decorator in TensorFlow that converts a Python function into a TensorFlow graph function.
# Benefits:
# Improved performance: By converting the function into a graph, TensorFlow can optimize the execution and reduce overhead.
# Eager execution: The decorated function can be executed in eager mode, which allows for more interactive development.
# Graph mode: The decorated function can also be executed in graph mode, which allows for more efficient execution.

In [5]:
@tf.function
def VGGNet(
    name: str,
    architecture: typing.List[ typing.Union[int, str] ],
    input_shape: typing.Tuple[int],
    classes: int = 1000
) -> Model:
    """
    Implementation of the VGGNet architecture.

    Arguments:
    name         -- name of the architecture
    architecture -- number of output channel per convolution layers in VGGNet
    input_shape  -- shape of the images of the dataset
    classes      -- integer, number of classes

    Returns:
    model        -- a Model() instance in Keras
    """

    # convert input shape into tensor
    X_input = Input(input_shape)

    # make convolution layers
    X = make_conv_layer(X_input, architecture)

    # flatten the output and make fully connected layers
    X = Flatten()(X)
    X = make_dense_layer(X, 4096)
    X = make_dense_layer(X, 4096)

    # classification layer
    X = Dense(units = classes, activation = "softmax")(X)

    model = Model(inputs = X_input, outputs = X, name = name)
    return model

In [6]:
VGG_types = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [7]:
model = VGGNet(name = "VGGNet11", architecture = VGG_types["VGG11"], input_shape=(224, 224, 3), classes = 1000)
model.summary()

Model: "VGGNet11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 224, 224, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 112, 112, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 56, 56, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 56, 56, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 28, 28, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 28, 28, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 28, 28, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 28, 28, 512)    │             

 Total params: 132,907,112 (507.00 MB)

 Trainable params: 132,885,224 (506.92 MB)

 Non-trainable params: 21,888 (85.50 KB)